In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate, get_compile_kwargs
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-10-22 16:20:59.419434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 16:20:59.948549: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [6]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/split_stage_data_unprocessed_100hz.nc")

data = xr.load_dataset(data_path)

In [8]:
data_path = Path("data/sat1/split_stage_data_weighted_mean.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/stage_data.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [4]:
model = SAT1LSTM(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

2023-10-22 14:17:59.302627: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 14:17:59.336521: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 14:17:59.336618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 14:17:59.339653: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 14:17:59.339725: I tensorflow/compile

In [5]:
results = k_fold_cross_validate(
    data,
    model,
    5,
    normalization_fn=norm_dummy,
    train_kwargs={"logs_path": Path("logs/rnn_performance"), "additional_name": "LSTM"},
)
print_results(results)

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/20


2023-10-22 14:18:06.071697: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_40/output/_23'
2023-10-22 14:18:06.913277: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-22 14:18:07.038530: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-22 14:18:07.3103

997/997 [==============================] - 24s 19ms/step - loss: 5.0674 - accuracy: 0.5979 - val_loss: 0.7298 - val_accuracy: 0.7266
Epoch 2/20
997/997 [==============================] - 18s 18ms/step - loss: 3.0158 - accuracy: 0.7699 - val_loss: 0.5610 - val_accuracy: 0.7923
Epoch 3/20
997/997 [==============================] - 18s 18ms/step - loss: 2.5350 - accuracy: 0.8080 - val_loss: 0.5600 - val_accuracy: 0.7985
Epoch 4/20
997/997 [==============================] - 19s 18ms/step - loss: 2.3101 - accuracy: 0.8240 - val_loss: 0.5350 - val_accuracy: 0.8113
Epoch 5/20
997/997 [==============================] - 18s 18ms/step - loss: 2.1745 - accuracy: 0.8338 - val_loss: 0.4931 - val_accuracy: 0.8241
Epoch 6/20
997/997 [==============================] - 18s 18ms/step - loss: 2.0273 - accuracy: 0.8427 - val_loss: 0.4931 - val_accuracy: 0.8285
Epoch 7/20
997/997 [==============================] - 18s 18ms/step - loss: 1.9138 - accuracy: 0.8543 - val_loss: 0.4870 - val_accuracy: 0.8364
Epo

In [4]:
model = SAT1GRU(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

2023-10-22 16:21:09.698241: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 16:21:09.723184: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 16:21:09.723259: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 16:21:09.725658: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 16:21:09.725724: I tensorflow/compile

In [9]:
results = k_fold_cross_validate(
    data,
    model,
    5,
    normalization_fn=norm_dummy,
    train_kwargs={"logs_path": Path("logs/rnn_performance"), "additional_name": "GRU"},
)
print_results(results)

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/20
997/997 [==============================] - 22s 19ms/step - loss: 5.4469 - accuracy: 0.5642 - val_loss: 0.6956 - val_accuracy: 0.7389
Epoch 2/20
997/997 [==============================] - 18s 18ms/step - loss: 2.8967 - accuracy: 0.7763 - val_loss: 0.5840 - val_accuracy: 0.7835
Epoch 3/20
997/997 [==============================] - 18s 18ms/step - loss: 2.5316 - accuracy: 0.8095 - val_loss: 0.5371 - val_accuracy: 0.8054
Epoch 4/20
997/997 [==============================] - 18s 18ms/step - loss: 2.2557 - accuracy: 0.8297 - val_loss: 0.5505 - val_accuracy: 0.8029
Epoch 5/20
997/997 [==============================] - 18s 18ms/step - loss: 2.1743 - accuracy: 0.8320 - val_loss: 0.5029 - val_accuracy: 0.8159
Epoch 6/20
997/997 [==============================] - 18s 18ms/step - loss: 2.0524 - accuracy: 0.8415 - val_loss: 0.4631 - val_accuracy: 0.8334
Epoch 7/20
997/997 [==============================] - 18s 18ms/step - loss: 1.96

In [5]:
results = k_fold_cross_validate(
    data,
    model,
    5,
    normalization_fn=norm_dummy,
    train_kwargs={
        "logs_path": Path("logs/rnn_performance"),
        "additional_name": "GRU_unprocessed_100hz",
    },
)
print_results(results)

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/20


2023-10-22 14:53:04.032444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-22 14:53:04.149721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-22 14:53:04.415836: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f3c685e4b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-22 14:53:04.415872: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-22 14:53:04.423895: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-22 14:53:04.553512: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

997/997 [==============================] - 25s 19ms/step - loss: 4.9127 - accuracy: 0.6156 - val_loss: 0.6585 - val_accuracy: 0.7577
Epoch 2/20
997/997 [==============================] - 18s 18ms/step - loss: 2.6348 - accuracy: 0.7977 - val_loss: 0.5018 - val_accuracy: 0.8142
Epoch 3/20
997/997 [==============================] - 18s 18ms/step - loss: 2.2485 - accuracy: 0.8264 - val_loss: 0.4627 - val_accuracy: 0.8347
Epoch 4/20
997/997 [==============================] - 18s 18ms/step - loss: 2.0469 - accuracy: 0.8438 - val_loss: 0.4725 - val_accuracy: 0.8327
Epoch 5/20
997/997 [==============================] - 18s 18ms/step - loss: 1.9306 - accuracy: 0.8514 - val_loss: 0.4590 - val_accuracy: 0.8369
Epoch 6/20
997/997 [==============================] - 18s 18ms/step - loss: 1.8091 - accuracy: 0.8596 - val_loss: 0.4366 - val_accuracy: 0.8421
Epoch 7/20
997/997 [==============================] - 18s 18ms/step - loss: 1.7482 - accuracy: 0.8658 - val_loss: 0.4265 - val_accuracy: 0.8449
Epo

In [5]:
results = k_fold_cross_validate(
    data,
    model,
    5,
    normalization_fn=norm_dummy,
    train_kwargs={
        "logs_path": Path("logs/rnn_performance"),
        "additional_name": "GRU_unprocessed_500hz",
    },
    fold_indices=[0, 1, 2],
)
print_results(results)

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/20


2023-10-22 16:07:39.630756: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-22 16:07:39.761187: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-22 16:07:40.020038: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557276176af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-22 16:07:40.020082: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-22 16:07:40.026451: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-22 16:07:40.141362: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

993/993 [==============================] - 34s 30ms/step - loss: 4.1895 - accuracy: 0.6903 - val_loss: 0.7000 - val_accuracy: 0.7362
Epoch 2/20
993/993 [==============================] - 28s 28ms/step - loss: 2.0137 - accuracy: 0.8524 - val_loss: 0.5274 - val_accuracy: 0.8185
Epoch 3/20
993/993 [==============================] - 29s 29ms/step - loss: 1.5111 - accuracy: 0.8916 - val_loss: 0.5564 - val_accuracy: 0.8133
Epoch 4/20
993/993 [==============================] - 28s 28ms/step - loss: 1.3417 - accuracy: 0.9025 - val_loss: 0.4816 - val_accuracy: 0.8447
Epoch 5/20
993/993 [==============================] - 28s 28ms/step - loss: 1.2298 - accuracy: 0.9111 - val_loss: 0.4890 - val_accuracy: 0.8382
Epoch 6/20
993/993 [==============================] - 28s 28ms/step - loss: 1.1310 - accuracy: 0.9160 - val_loss: 0.4780 - val_accuracy: 0.8440
Epoch 7/20
993/993 [==============================] - 28s 28ms/step - loss: 1.0790 - accuracy: 0.9212 - val_loss: 0.4950 - val_accuracy: 0.8460
Epo

In [5]:
results = k_fold_cross_validate(
    data,
    model,
    5,
    normalization_fn=norm_dummy,
    train_kwargs={
        "logs_path": Path("logs/rnn_performance"),
        "additional_name": "GRU_unprocessed_500hz",
    },
    fold_indices=[3, 4],
)
print_results(results)

Fold 4: test fold: ['0021' '0018' '0022' '0019' '0025']
Epoch 1/20


2023-10-22 16:21:25.250225: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-22 16:21:25.357348: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-22 16:21:25.612931: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a592205af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-22 16:21:25.612971: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-22 16:21:25.619206: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-22 16:21:25.733271: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

1004/1004 [==============================] - 34s 28ms/step - loss: 4.2030 - accuracy: 0.6881 - val_loss: 0.6539 - val_accuracy: 0.7453
Epoch 2/20
1004/1004 [==============================] - 28s 27ms/step - loss: 2.1117 - accuracy: 0.8484 - val_loss: 0.3547 - val_accuracy: 0.8771
Epoch 3/20
1004/1004 [==============================] - 28s 28ms/step - loss: 1.5871 - accuracy: 0.8855 - val_loss: 0.3145 - val_accuracy: 0.8865
Epoch 4/20
1004/1004 [==============================] - 28s 28ms/step - loss: 1.4015 - accuracy: 0.8987 - val_loss: 0.3722 - val_accuracy: 0.8776
Epoch 5/20
1004/1004 [==============================] - 28s 28ms/step - loss: 1.2894 - accuracy: 0.9066 - val_loss: 0.3403 - val_accuracy: 0.8826
Epoch 6/20
1004/1004 [==============================] - 28s 28ms/step - loss: 1.1755 - accuracy: 0.9142 - val_loss: 0.3132 - val_accuracy: 0.8959
Epoch 7/20
1004/1004 [==============================] - 28s 28ms/step - loss: 1.1478 - accuracy: 0.9171 - val_loss: 0.3254 - val_accura

In [6]:
accuracies = [
    0.84425,
    0.8938725490196079,
    0.8893649193548387,
    0.9014121338912134,
    0.8433734939759037
]
f1_scores = [
    0.8450428555225219,
    0.8923709281435693,
    0.8876558430686947,
    0.8985974413650201,
    0.8436775765453003
]
print(np.mean(accuracies))
print(np.mean(f1_scores))

0.8744546192483126
0.8734689289290213


In [6]:
model = SAT1seq2seqGRU(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

In [ ]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/")
)

In [ ]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    generator=SequentialSAT1DataGenerator,
)

In [11]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [6]:
model.save("models/gru")

INFO:tensorflow:Assets written to: models/gru/assets


INFO:tensorflow:Assets written to: models/gru/assets


In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 154, 30)]         0         
                                                                 
 masking (Masking)           (None, 154, 30)           0         
                                                                 
 gru (GRU)                   (None, 154, 64)           18432     
                                                                 
 gru_1 (GRU)                 (None, 154, 32)           9408      
                                                                 
 dense (Dense)               (None, 154, 5)            165       
                                                                 
Total params: 28005 (109.39 KB)
Trainable params: 28005 (109.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
